### Libraries Required

In [ ]:
#!pip install python-binance

#!pip install ta-lib 

#!pip install pandas-ta

In [ ]:
from binance import Client
import pandas as pd
import numpy as np
import talib as tb
import time
import pandas_ta as pta # to create supertrend indicator   
import plotly.graph_objs as go 

### Connect to Binance

In [ ]:
# API key and secret for the testnet

key = ''
secret = ''

In [ ]:
# Sending request to Binance Testnet to connect to Python

client = Client(api_key=key, api_secret=secret, testnet=True)

### Trading bot - get live data

In [ ]:
# Getting live data in a data frame format from Binance API and the date into an organized
# and readable DataFrame.

def getdata(symbol, interval, past):
    frame = pd.DataFrame(client.get_historical_klines(symbol, interval, past + ' min ago UTC'))
    frame = frame.iloc[:, :5]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame

# Retrieving and reading the data into a DataFrame for the given symbol, interval, and 
# past time period.

df = getdata('BTCUSDT', '1m', '100')

df

### Calculate Supertrend Indicator

In [ ]:
# Calculating the SuperTrend indicator using the pandas_ta library

spt = pta.supertrend(df['High'], df['Low'], df['Close'], 4, 2)

# Concatenating the SuperTrend DataFrame with the original DataFrame

df = pd.concat([df,spt], axis = 1)

df

### Define Strategy

In [ ]:
# Defining strategy based on Supertrend Indicator

def strategy(pair, interval, past, qty, open_position = False):
    df = getdata(pair, interval, past)
    spt = pta.supertrend(df['High'], df['Low'], df['Close'], 4, 2)
    df = pd.concat([df, spt], axis=1)
    print(f'Current Close Price is ' + str(df.Close.iloc[-1]))
    print(f'Current Supertrend value is ' + str(df['SUPERT_4_2.0'].iloc[-1]))
    if df['SUPERTd_4_2.0'].iloc[-1] == 1: # Checking if the SuperTrend direction indicator is 1 (indicating an uptrend)
        order = client.create_order(symbol = pair, side = 'BUY', type = 'MARKET', quantity = qty)
        print(order)
        open_position = True
    while open_position:
        df = getdata(pair, interval, past)
        spt = pta.supertrend(df['High'], df['Low'], df['Close'], 4, 2)
        df = pd.concat([df, spt], axis=1)
        print(f'Current Close Price is ' + str(df.Close.iloc[-1]))
        print(f'Current Supertrend value is ' + str(df['SUPERT_4_2.0'].iloc[-1]))
        if df['SUPERTd_4_2.0'].iloc[-1] == -1: # Checking if the SuperTrend direction indicator is -1 (indicating a downtrend)
            order = client.create_order(symbol = pair, side = 'SELL', type = 'MARKET', quantity = qty)
            print(order)
            open_position = False
            break

### Supertrend Indicator Visualization

In [ ]:

# Creating figure
fig = go.Figure()

# Adding candlestick trace
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close'],
                             name='Market Data'))

# Adding a scatter trace for the SuperTrend Up values
fig.add_trace(go.Scatter(x=df.index,
                         y=df['SUPERTs_4_2.0'],
                         line=dict(color='red', width=1.5),
                         name='SPT Up'))

# Adding a scatter trace for the SuperTrend Down values
fig.add_trace(go.Scatter(x=df.index,
                         y=df['SUPERTl_4_2.0'],
                         line=dict(color='blue', width=1.5),
                         name='SPT Down'))

# Updating layout
fig.update_layout(
    title='ETHUSDT Market Data',
    yaxis_title='ETHUSDT (US Dollars)',
    xaxis_title='Date',
    xaxis_rangeslider_visible=True,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="minute", stepmode="backward"),
                dict(count=15, label="15m", step="minute", stepmode="backward"),
                dict(count=1, label="1h", step="hour", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# Updating y-axis with dollar sign prefix
fig.update_yaxes(tickprefix="$")

# Showing figure
fig.show()

### Golden Cross Strategy

In [ ]:
# Calculating moving averages
df['MA5'] = df['Close'].rolling(5).mean()
df['MA20'] = df['Close'].rolling(20).mean()

# Creating a figure
fig = go.Figure()

# Adding candlestick trace
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close'],
                             name='Market Data'))

# Short-term moving average trace
fig.add_trace(go.Scatter(x=df.index,
                         y=df['MA5'],
                         line=dict(color='orange', width=1.5),
                         name='Short Term MA'))

# Long-term moving average trace
fig.add_trace(go.Scatter(x=df.index,
                         y=df['MA20'],
                         line=dict(color='purple', width=1.5),
                         name='Long Term MA'))

# Updating layout
fig.update_layout(
    title='ETHUSDT Market Data with Golden Cross',
    yaxis_title='ETHUSDT (US Dollars)',
    xaxis_title='Date',
    xaxis_rangeslider_visible=True,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="minute", stepmode="backward"),
                dict(count=15, label="15m", step="minute", stepmode="backward"),
                dict(count=1, label="1h", step="hour", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# Updating y-axis with dollar sign prefix
fig.update_yaxes(tickprefix="$")

# Showing figure
fig.show()

### Bollinger Band

In [ ]:
# Parameters for Bollinger Bands
window = 30
no_of_std = 2

# Calculating moving average
df['SMA'] = df['Close'].rolling(window=window).mean()

# Calculating rolling standard deviation
df['STD'] = df['Close'].rolling(window=window).std()

# Calculating upper and lower Bollinger Bands
df['Upper_Band'] = df['SMA'] + (df['STD'] * no_of_std)
df['Lower_Band'] = df['SMA'] - (df['STD'] * no_of_std)

# Creating a figure
fig = go.Figure()

# Adding candlestick trace
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close'],
                             name='market data'))

# Adding Bollinger Bands traces
fig.add_trace(go.Scatter(x=df.index,
                         y=df['Upper_Band'],
                         line=dict(color='red', width=1.5),
                         name='Upper Bollinger Band'))

fig.add_trace(go.Scatter(x=df.index,
                         y=df['SMA'],
                         line=dict(color='blue', width=1.5),
                         name='20-day SMA'))

fig.add_trace(go.Scatter(x=df.index,
                         y=df['Lower_Band'],
                         line=dict(color='red', width=1.5),
                         name='Lower Bollinger Band'))

# Updating layout
fig.update_layout(
    title='ETHUSDT Market Data with Bollinger Bands',
    yaxis_title='ETHUSDT (US Dollars)',
    xaxis_title='Date',
    xaxis_rangeslider_visible=True,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="minute", stepmode="backward"),
                dict(count=15, label="15m", step="minute", stepmode="backward"),
                dict(count=1, label="1h", step="hour", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# Updating y-axis with dollar sign prefix
fig.update_yaxes(tickprefix="$")

# Showing figure
fig.show()

### Strategy Indicators

In [ ]:
# Recalculating some fields to have the most updated values
# Calculating moving averages
df['MA5'] = df['Close'].rolling(5).mean()
df['MA20'] = df['Close'].rolling(20).mean()

# Parameters for Bollinger Bands
window = 20
no_of_std = 2

# Calculate moving average
df['SMA'] = df['Close'].rolling(window=window).mean()

# Calculate rolling standard deviation
df['STD'] = df['Close'].rolling(window=window).std()

# Calculate upper and lower Bollinger Bands
df['Upper_Band'] = df['SMA'] + (df['STD'] * no_of_std)
df['Lower_Band'] = df['SMA'] - (df['STD'] * no_of_std)

# Creating a figure
fig = go.Figure()

# Adding candlestick trace
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close'],
                             name='Market Data'))

# Adding SPT Up trace
fig.add_trace(go.Scatter(x=df.index,
                         y=df['SUPERTs_4_2.0'],
                         line=dict(color='red', width=1.5),
                         name='SPT Up'))

# Adding SPT Down trace
fig.add_trace(go.Scatter(x=df.index,
                         y=df['SUPERTl_4_2.0'],
                         line=dict(color='blue', width=1.5),
                         name='SPT Down'))

# Short-term moving average trace
fig.add_trace(go.Scatter(x=df.index,
                         y=df['MA5'],
                         line=dict(color='orange', width=1.5),
                         name='Short Term MA'))

# Long-term moving average trace
fig.add_trace(go.Scatter(x=df.index,
                         y=df['MA20'],
                         line=dict(color='purple', width=1.5),
                         name='Long Term MA'))

# Adding Bollinger Bands traces
fig.add_trace(go.Scatter(x=df.index,
                         y=df['Upper_Band'],
                         line=dict(color='red', width=1.5),
                         name='Upper Bollinger Band'))

fig.add_trace(go.Scatter(x=df.index,
                         y=df['SMA'],
                         line=dict(color='purple', width=1.5),
                         name='20-day SMA'))

fig.add_trace(go.Scatter(x=df.index,
                         y=df['Lower_Band'],
                         line=dict(color='green', width=1.5),
                         name='Lower Bollinger Band'))

# Updating layout
fig.update_layout(
    title='ETHUSDT Market Data with Indicators',
    yaxis_title='ETHUSDT (US Dollars)',
    xaxis_title='Date',
    xaxis_rangeslider_visible=True,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="minute", stepmode="backward"),
                dict(count=15, label="15m", step="minute", stepmode="backward"),
                dict(count=1, label="1h", step="hour", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# Updating y-axis with dollar sign prefix
fig.update_yaxes(tickprefix="$")

# Showing figure
fig.show()

### Run Trading Bot

In [ ]:
# Continuously running the trading strategy in an infinite loop

while True:
    
    # Executing the trading strategy for the specified symbol, interval, past time period, and quantity
    strategy('BTCUSDT', '1m', '30', 5)
    time.sleep(1)